In [1]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import glob
import os

# Conectar ao PostgreSQL (NeonDB)
conn = psycopg2.connect(
    dbname="neondb",  # Substitua com o nome do seu banco de dados
    user="neondb_owner",  # Substitua com o nome de usuário
    password="npg_ayeAjRZ8sPD7",  # Substitua com sua senha
    host="ep-rough-shape-a85bihtu-pooler.eastus2.azure.neon.tech",  # Substitua com o host se necessário
    port="5432"  # Substitua com a porta se necessário
)

try:
    conn = psycopg2.connect(
        dbname="neondb",
        user="neondb_owner",  # Substitua com o nome de usuário
        password="npg_ayeAjRZ8sPD7",  # Substitua com sua senha
        host="ep-rough-shape-a85bihtu-pooler.eastus2.azure.neon.tech",  # Substitua com o host se necessário
        port="5432"
    )
    cur = conn.cursor()
    cur.execute("SELECT version();")
    versao = cur.fetchone()
    print(f"Conexão bem-sucedida! Versão do PostgreSQL: {versao[0]}")
    cur.close()
    conn.close()
except Exception as e:
    print(f"Erro ao conectar ao banco de dados: {e}")


Conexão bem-sucedida! Versão do PostgreSQL: PostgreSQL 17.4 on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit


In [15]:
import pandas as pd
import os

# Definir o caminho do arquivo
caminho_arquivo ='/content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/04-XP/Input/CreditoXP.xlsx'

# Lista fixa com os nomes das colunas
nomes_colunas = [
    'data_lanc', 'estabelecimento', 'portador', 'valor', 'parcela', 'data_vencimento', 'idcategoria', 'descricao', 'transferencia'
]

# Ler o arquivo Excel e armazenar em um DataFrame
df = pd.read_excel(caminho_arquivo)

# Remover linhas completamente vazias
df = df.dropna(how='all')

# Renomear colunas (substitua pelos nomes desejados)
novos_nomes = {
    'Data': 'data_lanc',
    'Estabelecimento': 'estabelecimento',
    'Portador': 'portador',
    'Valor': 'valor',
    'Parcela': 'parcela',
    'Data Vencimento': 'data_vencimento',
    'idCategoria': 'idcategoria',
    'Descrição': 'descricao',
    'Transferencia': 'transferencia'
}

df = df.rename(columns=novos_nomes)

# Remover colunas indesejadas (substitua pelos nomes das colunas a excluir)
colunas_para_remover = ['transferencia', 'portador', 'transferencia']
df = df.drop(columns=colunas_para_remover, errors='ignore')

df['data_carga'] = pd.to_datetime('today').normalize()  # 'normalize' para deixar a hora como 00:00:00

# Exibir as primeiras linhas do DataFrame
df



,data_lanc,estabelecimento,valor,parcela,data_vencimento,idcategoria,descricao,data_carga
0,2021-12-21,SERINGUEIRAS RESTAURAN,74.80,-,2022-01-05,GRSA04,refeição GRSA,2025-03-30
1,2021-12-22,SERINGUEIRAS RESTAURAN,89.30,-,2022-01-05,GRSA04,refeição GRSA,2025-03-30
2,2021-12-27,POSTO SAO JOAQUIM,100.00,-,2022-02-05,GRSA02,Combustível GRSA,2025-03-30
3,2021-12-27,SERINGUEIRAS RESTAURAN,31.90,-,2022-02-05,GRSA04,refeição GRSA,2025-03-30
4,2021-12-28,SERINGUEIRAS RESTAURAN,157.60,-,2022-02-05,GRSA04,refeição GRSA,2025-03-30
...,...,...,...,...,...,...,...,...
287,2025-03-27,Encargos de Refinanciamento,2.80,de 1,2025-04-05,Imp09,NaN,2025-03-30
288,2025-03-27,Juros de Mora,0.31,de 1,2025-04-05,Imp09,NaN,2025-03-30
290,2024-06-08,ACADEMIA FOUR SWI,158.40,11 de 12,2025-05-05,CPes01,academia,2025-03-30
291,2024-07-21,AMAZON MARKETPLACE,361.90,10 de 10,2025-05-05,Outr12,tv senira,2025-03-30


In [13]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd

# Credenciais do banco de dados
dbname = "neondb"
user = "neondb_owner"
password = "npg_ayeAjRZ8sPD7"
host = "ep-rough-shape-a85bihtu-pooler.eastus2.azure.neon.tech"
port = "5432"

# Criar a string de conexão com o PostgreSQL
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

# Criar a conexão com o banco de dados
engine = create_engine(connection_string)

In [16]:
from sqlalchemy import text

# Função para excluir dados antigos e inserir novos, com relatórios
def excluir_e_inserir(df, engine):
    with engine.connect() as conn:
        # Verificar se a tabela existe no banco
        check_table_query = text("""
            SELECT EXISTS (
                SELECT FROM information_schema.tables
                WHERE table_schema = 'fluxo'
                AND table_name = 'xp'
            );
        """)
        table_exists = conn.execute(check_table_query).scalar()

        if table_exists:
            # Se a tabela existir, deletar os dados e gravar novamente
            delete_query = text(f"DELETE FROM fluxo.xp;")
            conn.execute(delete_query)
            conn.commit()
            df.to_sql('xp', engine, schema='fluxo', if_exists='append', index=False)
        else:
            # Se a tabela não existir, considera que nenhum arquivo foi importado ainda
            df.to_sql('xp', engine, schema='fluxo', if_exists='append', index=False)

    # Relatório de execução
    print(f"Processamento concluído.")

# Chamar a função para excluir dados antigos e inserir os novos
excluir_e_inserir(df, engine)


Processamento concluído.


In [ ]:
#df_filtrado.info()
#df_filtrado.describe()
#df_filtrado['num_doc'].unique()

#df_filtrado['num_doc'].duplicated().any()


df_filtrado[df_filtrado['num_doc'].duplicated(keep=False)]


,data_lanc,descricao,num_doc,valor,idcategoria,observacao,nome_arquivo,data_carga
0,2019-02-25,Saldo Anterior,0,140.44,NaN,NaN,BB_extrato.xlsx,2025-03-30
2,2022-03-30,Saldo Anterior,0,20.44,NaN,NaN,BB_extrato.xlsx,2025-03-30
3,2022-03-31,S A L D O,0,20.44,NaN,NaN,BB_extrato.xlsx,2025-03-30
5,2022-04-22,Pix - Enviado,42201,-550.00,Outr03,não identificado,BB_extrato.xlsx,2025-03-30
6,2022-04-22,Resgate Poupança,148,342.36,Rec09,resgate Poupança,BB_extrato.xlsx,2025-03-30
...,...,...,...,...,...,...,...,...
1135,2025-03-18,Pix - Enviado - 18/03 16:02 Ana Paula Rodrigue...,31801,-10.32,Outr03,NaN,BB_extrato.xlsx,2025-03-30
1136,2025-03-18,Pix - Enviado - 18/03 19:14 Organizacao F N Ltda,31802,-166.59,Outr03,NaN,BB_extrato.xlsx,2025-03-30
1137,2025-03-20,Pix - Enviado - 20/03 12:31 Drogaria Tranquili...,32001,-178.00,CPes06,NaN,BB_extrato.xlsx,2025-03-30
1145,2025-03-27,Pix - Enviado - 27/03 22:19 Maria Fatima Santo...,32701,-10.00,Outr03,NaN,BB_extrato.xlsx,2025-03-30
